# Problem statement: Help the bank find out why their customers are leaving

## Import Dataset and necessary libraries

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Import the necessary Keras libraries
import keras
#Sequential module for initializing neural network
#Dense module to build layers of ANN
from keras.models import Sequential
from keras.layers import Dense


from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
#Import sample churn modelling data for a bank
bank_data=pd.read_csv('Churn_Modelling.csv')
bank_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = bank_data.iloc[:, 3:13].values
y = bank_data.iloc[:, 13].values
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [4]:
X[0]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

## Preprocess data

In [5]:
#Encoding categorical variable data
labelencoder_X_1=LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])
X

array([[619, 0, 'Female', ..., 1, 1, 101348.88],
       [608, 2, 'Female', ..., 0, 1, 112542.58],
       [502, 0, 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 0, 'Female', ..., 0, 1, 42085.58],
       [772, 1, 'Male', ..., 1, 0, 92888.52],
       [792, 0, 'Female', ..., 1, 0, 38190.78]], dtype=object)

### Encode the categorical variables , remove dummy variables (if two or more categories are present)

In [6]:
#Encoding categorical variable data
labelencoder_X_2=LabelEncoder()
X[:,2]=labelencoder_X_2.fit_transform(X[:,2])
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [7]:
onehotencoder=OneHotEncoder(categorical_features=[1])
X=onehotencoder.fit_transform(X).toarray()
X

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [8]:
#Dummy variable trap for country column - remove one of three labels
X=X[:,1:]
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

### Split data into training and validation set

In [9]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

In [11]:
X_train.shape

(8000, 11)

## Training ANN model

In [12]:
classifier=Sequential()

In [13]:
#Adding the input layer and the first hidden layer
#Calculate output dim value by taking avg. of number of variables of input(11) and output(1)
classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu',input_dim=11))

In [14]:
#Adding the second hidden layer
#Calculate output dim value by taking avg. of number of variables of input(11) and output(1)
classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu'))

In [15]:
#Adding the second hidden layer
#Calculate output dim value by taking avg. of number of variables of input(11) and output(1)
#Choose softmax activation function while dealing with dependent variable of more than two categories
classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))

In [16]:
# Compile the ANN network
#Use category_crossentropy for more than 2 categories of dependent variables
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [17]:
# Predicting the test results
#Using gradient descent
classifier.fit(X_train,y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 2s 215us/step - loss: 0.4919 - acc: 0.7982
Epoch 2/100
8000/8000 [==============================] - 1s 165us/step - loss: 0.4108 - acc: 0.8226
Epoch 3/100
8000/8000 [==============================] - 2s 192us/step - loss: 0.3965 - acc: 0.8276
Epoch 4/100
8000/8000 [==============================] - 1s 173us/step - loss: 0.3867 - acc: 0.8305
Epoch 5/100
8000/8000 [==============================] - 1s 158us/step - loss: 0.3799 - acc: 0.8309
Epoch 6/100
8000/8000 [==============================] - 1s 163us/step - loss: 0.3748 - acc: 0.8409
Epoch 7/100
8000/8000 [==============================] - 1s 154us/step - loss: 0.3707 - acc: 0.8464
Epoch 8/100
8000/8000 [==============================] - 1s 162us/step - loss: 0.3680 - acc: 0.8479
Epoch 9/100
8000/8000 [==============================] - 1s 160us/step - loss: 0.3655 - acc: 0.8496
Epoch 10/100
8000/8000 [==============================] - 1s 157us/step - loss: 0.3627 - acc: 0.8536

## Measure accuracy of the model

In [18]:
#Set threshold to map values to specific bins
y_pred=classifier.predict(X_test)
y_pred = (y_pred>0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [19]:
#Make the confusion matrix
cm=confusion_matrix(y_test,y_pred)

In [20]:
cm

array([[1551,   44],
       [ 220,  185]], dtype=int64)

In [23]:
cm[1][1]

185

In [24]:
# accuracy= (TP+TN)/total count
accuracy=(cm[0][0]+cm[1][1])/2000
accuracy

0.868

### As can be seen, the network converged around 87% accuracy which is not bad considering all the elementary parameter values and basic models we've used. Will improve the model in next few iterations 